In [1]:
# импортируем библиотеки numpy и pandas
import numpy as np
import pandas as pd
# импортируем функцию train_test_split(), с помощью
# которой разбиваем данные на обучающие и тестовые
from sklearn.model_selection import train_test_split
# импортируем функцию boxcox() для выполнения
# преобразования Бокса-Кокса
from scipy.stats import boxcox
# импортируем класс LogisticRegression для построения
# логистической регрессии
from sklearn.linear_model import LogisticRegression

In [2]:
# записываем CSV-файл в объект DataFrame
data = pd.read_csv('Data/Verizon_miss_dubl.csv', sep=';')

In [3]:
# смотрим первые 5 наблюдений
data.head(5)

,longdist,internat,local,int_disc,billtype,pay,age,gender,marital,children,income,churn
0,"27,09",0,"39,74",Нет,Бюджетный,CC,35.0,Женский,Женат,0.0,77680,0
1,NaN,0,"46,31",Нет,NaN,NaN,53.0,Мужской,Одинокий,1.0,"37111,5",0
2,"23,76",0,NaN,NaN,Бюджетный,Auto,NaN,Женский,NaN,1.0,"37079,4",0
3,"9,4",NaN,"13,9",Нет,NaN,CH,NaN,Мужской,Одинокий,NaN,81997,0
4,"14,15",0,"108,43",Да,Бесплатный,Auto,39.0,Женский,Одинокий,0.0,"16829,6",0


In [4]:
# смотрим типы переменных
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4431 entries, 0 to 4430
Data columns (total 12 columns):
longdist    4430 non-null object
internat    4427 non-null object
local       4428 non-null object
int_disc    4430 non-null object
billtype    4427 non-null object
pay         4429 non-null object
age         4428 non-null float64
gender      4430 non-null object
marital     4427 non-null object
children    4430 non-null float64
income      4430 non-null object
churn       4431 non-null int64
dtypes: float64(2), int64(1), object(9)
memory usage: 415.5+ KB


In [5]:
# заменяем запятые на точки и преобразуем в тип float
for i in ['longdist', 'internat', 'local', 'income']:
    data[i] = data[i].str.replace(',', '.').astype('float')

In [6]:
# создаем список категориальных переменных
cat_cols = data.select_dtypes(include=['object']).columns.tolist()
# смотрим уникальные значения этих переменных
for i in cat_cols:
    print(i, data[i].unique())

int_disc ['Нет' nan 'Да']
billtype ['Бюджетный' nan 'Бесплатный']
pay ['CC' nan 'Auto' 'CH' 'CD']
gender ['Женский' 'Мужской' nan 'Женский&*' 'Мужской&*']
marital ['Женат' 'Одинокий' nan '_Одинокий' '_Женат' 'Же&нат']


In [7]:
# удаляем лишние символы в категориях переменных
# gender и marital
for i in ['gender', 'marital']:
    data[i] = data[i].str.replace('[*&_]', '')
    
# проверяем
for i in ['gender', 'marital']:
    print(i, data[i].unique())

gender ['Женский' 'Мужской' nan]
marital ['Женат' 'Одинокий' nan]


In [8]:
# удаляем дубли на месте, оставляя первое
# встретившееся наблюдение в паттерне дубля
data.drop_duplicates(subset=None, keep='first', 
                     inplace=True)
# смотрим, сколько наблюдений осталось
len(data)

1487

In [9]:
# смотрим частоты по категориальным переменным, 
# чтобы выявить редкие категории
for i in cat_cols:
    print(i)
    print('')
    print(data[i].value_counts(dropna=False))

int_disc

Нет    1025
Да      461
NaN       1
Name: int_disc, dtype: int64
billtype

Бюджетный     752
Бесплатный    731
NaN             4
Name: billtype, dtype: int64
pay

CC      856
CH      329
Auto    298
CD        2
NaN       2
Name: pay, dtype: int64
gender

Женский    751
Мужской    735
NaN          1
Name: gender, dtype: int64
marital

Женат       877
Одинокий    606
NaN           4
Name: marital, dtype: int64


In [10]:
# заменяем редкую категорию модой
data.at[data['pay'] == 'CD', 'pay'] = 'CC'

In [11]:
# пишем функцию, создающую парные взаимодействия
def make_conj(df, feature1, feature2):
    df[feature1 + "_" + feature2] = df[feature1].astype('object') + " + " + df[feature2].astype('object')

In [12]:
# применяем функцию
make_conj(data, 'gender', 'marital')

In [13]:
# поделим возраст на длительность междугородних звонков в минутах
data['ratio'] = data['age'] / data['longdist']
# заменяем бесконечные значения на 1
data['ratio'].replace([np.inf, -np.inf], 1, inplace=True)

In [14]:
# поделим длительность междугородних звонков в минутах на
# длительность международных звонков в минутах
data['ratio2'] = data['longdist'] / data['internat']
# заменяем бесконечные значения на 0
data['ratio2'].replace([np.inf, -np.inf], 0, inplace=True)

In [15]:
# поделим доход на возраст
data['ratio3'] = data['income'] / data['age']
# заменяем бесконечные значения на 0
data['ratio3'].replace([np.inf, -np.inf], 0, inplace=True)

In [16]:
# поделим возраст на количество детей
data['ratio4'] = data['age'] / data['children']
# заменяем бесконечные значения на 0
data['ratio4'].replace([np.inf, -np.inf], 0, inplace=True)

In [17]:
# разбиваем данные на обучающую и тестовую выборки
train, test, y_train, y_test = train_test_split(data.drop('churn', axis=1), 
                                                data['churn'],
                                                test_size=0.3,
                                                stratify=data['churn'],
                                                random_state=42)

In [18]:
# взглянем на пропуски
print(train.isnull().sum())
print(test.isnull().sum())

longdist            0
internat            3
local               3
int_disc            1
billtype            3
pay                 1
age                 2
gender              1
marital             3
children            1
income              0
gender_marital      3
ratio               2
ratio2            125
ratio3              2
ratio4              2
dtype: int64
longdist           1
internat           1
local              0
int_disc           0
billtype           1
pay                1
age                1
gender             0
marital            1
children           0
income             1
gender_marital     1
ratio              2
ratio2            44
ratio3             1
ratio4             1
dtype: int64


In [19]:
# заменяем пропуски в количественных переменных средними
numerical_columns = train.dtypes[train.dtypes != 'object'].index 
for i in numerical_columns:
    train[i].fillna(train[i].mean(), inplace=True)
    test[i].fillna(train[i].mean(), inplace=True)

In [20]:
# заменяем пропуски в категориальных переменных модой
categorical_columns = train.dtypes[train.dtypes == 'object'].index
for i in categorical_columns:
    train[i].fillna(train[i].value_counts().index[0], inplace=True)
    test[i].fillna(train[i].value_counts().index[0], inplace=True)

In [21]:
# задаем точки, в которых будут находится границы категорий 
# будущей переменной agecat
bins = [-np.inf, 30, 50, np.inf]
# осуществляем биннинг переменной age и записываем
# результаты в новую переменную agecat
train['agecat'] = pd.cut(train['age'], bins).astype('object')
test['agecat'] = pd.cut(test['age'], bins).astype('object')

In [22]:
# подготавливаем данные перед преобразованием Бокса-Кокса
# (данные должны быть положительными)
train.replace({0: 0.5}, inplace=True)
test.replace({0: 0.5}, inplace=True)

In [23]:
# выполняем преобразование Бокса-Кокса
for i in numerical_columns:  
    train[i], fitted_lambda = boxcox(train[i])     
    test[i] = boxcox(test[i], fitted_lambda)   

In [24]:
# выполняем стандартизацию количественных переменных
train_copy = train.copy()
for i in numerical_columns:    
    train[i] = (train[i] - train[i].mean()) / train[i].std()
    test[i] = (test[i] - train_copy[i].mean()) / train_copy[i].std()

In [25]:
# выполняем дамми-кодирование
X_train = pd.get_dummies(train)
X_test = pd.get_dummies(test)

In [26]:
# создаем экземпляр класса LogisticRegression
logreg = LogisticRegression(solver='lbfgs', max_iter=200)
# обучаем модель
logreg.fit(X_train, y_train)
# оцениваем правильность модели на обучающих данных
print('Правильность на обучающей выборке: {:.3f}'.format(
    logreg.score(X_train, y_train)))
# оцениваем правильность модели на тестовых данных
print('Правильность на тестовой выборке: {:.3f}'.format(
    logreg.score(X_test, y_test)))

Правильность на обучающей выборке: 0.816
Правильность на тестовой выборке: 0.799
